# KPI Card Data Extraction Notebook

This notebook replicates the data extraction process for each Level 1 KPI card in the hospital dashboard.

It demonstrates:
1. How data is retrieved from the DuckDB database
2. How KPI values are calculated
3. How benchmark data is fetched
4. How the calculation displays are generated

In [62]:
# Import required libraries
import duckdb
import pandas as pd
import sys
from pathlib import Path

# Add project root to path
sys.path.insert(0, str(Path.cwd()))

from data.data_manager import HospitalDataManager
from utils.formatting import format_number_compact
from kpi_hierarchy_config import KPI_METADATA

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1. Initialize Data Manager and Select Hospital

In [79]:
# Initialize data manager
data_manager = HospitalDataManager()

# Select a test hospital (you can change this)
test_ccn = '014000'
print(f"Testing with Hospital CCN: {test_ccn}")

# Get hospital metadata
hospital_type = data_manager.classify_hospital_type(test_ccn)
state_code = str(test_ccn)[:2]
print(f"Hospital Type: {hospital_type}")
print(f"State Code: {state_code}")

[OK] Using optimized database with pre-computed KPIs: data/hospital_analytics.duckdb
[OK] Connected to worksheet database: data/hospital_worksheets.duckdb
     Found 26 worksheet tables
Testing with Hospital CCN: 014000
Hospital Type: Psychiatric
State Code: 01


## 2. Get KPI Data for All Years

In [80]:
# Get KPI data (5-year history)
kpi_data = data_manager.calculate_kpis(test_ccn)

print(f"\nKPI Data Shape: {kpi_data.shape}")
print(f"Years available: {sorted(kpi_data['Fiscal_Year'].unique())}")
print(f"\nColumns available ({len(kpi_data.columns)}):")
print(kpi_data.columns.tolist())


KPI Data Shape: (2, 37)
Years available: [np.int64(2021), np.int64(2023)]

Columns available (37):
['Provider_Number', 'Fiscal_Year', 'Current_Assets', 'Fixed_Assets', 'Total_Assets', 'Current_Liabilities', 'Total_Liabilities', 'Fund_Balance', 'Cash_And_Securities', 'Accounts_Receivable', 'Inpatient_Revenue', 'Outpatient_Revenue', 'Total_Revenue', 'Total_Operating_Expenses', 'Net_Income', 'Operating_Margin_Pct', 'Net_Margin_Pct', 'Total_Margin_Pct', 'Current_Ratio', 'Days_Cash_On_Hand', 'Working_Capital', 'Outpatient_Revenue_Pct', 'Inpatient_Revenue_Pct', 'Asset_Turnover_Ratio', 'Fixed_Asset_Turnover', 'AR_Days', 'Operating_Expense_Ratio', 'Debt_to_Equity_Ratio', 'Equity_Ratio_Pct', 'Debt_Ratio_Pct', 'Return_on_Assets_Pct', 'Return_on_Equity_Pct', 'Revenue_Growth_Pct', 'Adjusted_Discharges', 'Operating_Expense_per_Adjusted_Discharge', 'Medicare_CCR', 'Bad_Debt_Charity_Pct']


In [81]:
# Display first few rows
print("\nKPI Data (most recent 3 years):")
kpi_data.head(3)


KPI Data (most recent 3 years):


,Provider_Number,Fiscal_Year,Current_Assets,Fixed_Assets,Total_Assets,Current_Liabilities,Total_Liabilities,Fund_Balance,Cash_And_Securities,Accounts_Receivable,Inpatient_Revenue,Outpatient_Revenue,Total_Revenue,Total_Operating_Expenses,Net_Income,Operating_Margin_Pct,Net_Margin_Pct,Total_Margin_Pct,Current_Ratio,Days_Cash_On_Hand,Working_Capital,Outpatient_Revenue_Pct,Inpatient_Revenue_Pct,Asset_Turnover_Ratio,Fixed_Asset_Turnover,AR_Days,Operating_Expense_Ratio,Debt_to_Equity_Ratio,Equity_Ratio_Pct,Debt_Ratio_Pct,Return_on_Assets_Pct,Return_on_Equity_Pct,Revenue_Growth_Pct,Adjusted_Discharges,Operating_Expense_per_Adjusted_Discharge,Medicare_CCR,Bad_Debt_Charity_Pct
0,14000,2023,11146696.0,5948368.0,88907685.0,29438442.0,44157663.0,59469243.0,-81142.0,5532143.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.378644,NaN,-18.291746,NaN,NaN,NaN,NaN,NaN,NaN,0.742529,66.888754,49.666869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14000,2021,7711316.0,8003668.0,85034937.0,21855864.0,32783796.0,63179073.0,-111644.0,3789301.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.352826,NaN,-14.144548,NaN,NaN,NaN,NaN,NaN,NaN,0.518903,74.297783,38.553325,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Level 1 KPIs Overview

In [82]:
# Define the 6 Level 1 KPIs
LEVEL_1_KPIS = [
    'Net_Income_Margin',
    'AR_Days',
    'Operating_Expense_per_Adjusted_Discharge',
    'Medicare_CCR',
    'Bad_Debt_Charity_Pct',
    'Current_Ratio'
]

# Get metadata for each KPI
print("Level 1 KPIs Configuration:")
print("=" * 100)
for kpi_key in LEVEL_1_KPIS:
    meta = KPI_METADATA.get(kpi_key, {})
    print(f"\n{kpi_key}:")
    print(f"  Name: {meta.get('name', 'N/A')}")
    print(f"  Category: {meta.get('category', 'N/A')}")
    print(f"  Unit: {meta.get('unit', 'N/A')}")
    print(f"  Format: {meta.get('format', 'N/A')}")
    print(f"  Higher is Better: {meta.get('higher_is_better', 'N/A')}")
    print(f"  Formula: {meta.get('formula_description', 'N/A')}")

Level 1 KPIs Configuration:

Net_Income_Margin:
  Name: Net Income Margin
  Category: Profitability
  Unit: %
  Format: .1f
  Higher is Better: True
  Formula: (Net Income) ÷ (Total Revenue)

AR_Days:
  Name: Days in Net Patient AR
  Category: Revenue Cycle
  Unit: days
  Format: .0f
  Higher is Better: False
  Formula: (Net Patient AR) ÷ (Net Patient Revenue / 365)

Operating_Expense_per_Adjusted_Discharge:
  Name: Operating Expense per Adjusted Discharge
  Category: Cost Management
  Unit: $
  Format: ,.0f
  Higher is Better: False
  Formula: (Total Operating Expenses) ÷ (Adjusted Discharges)

Medicare_CCR:
  Name: Medicare Cost-to-Charge Ratio
  Category: Efficiency
  Unit: ratio
  Format: .3f
  Higher is Better: False
  Formula: (Total Costs) ÷ (Total Charges)

Bad_Debt_Charity_Pct:
  Name: Bad Debt + Charity %
  Category: Revenue Cycle
  Unit: %
  Format: .1f
  Higher is Better: False
  Formula: (Bad Debt + Charity Care) ÷ (Net Revenue - Provisions)

Current_Ratio:
  Name: Current

## 4. Card #1: Net Income Margin

In [83]:
print("=" * 100)
print("CARD #1: NET INCOME MARGIN")
print("=" * 100)

# Get latest year
latest_year = kpi_data['Fiscal_Year'].max()
print(f"\nLatest Year: {latest_year}")

# Extract relevant columns
nim_data = kpi_data[['Fiscal_Year', 'Net_Income', 'Total_Revenue', 'Net_Margin_Pct']].copy()
nim_data = nim_data.sort_values('Fiscal_Year', ascending=False)

print("\nNet Income Margin - 5 Year History:")
print(nim_data)

# Get current year values
current_net_income = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Net_Income'].iloc[0]
current_revenue = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Total_Revenue'].iloc[0]
current_margin = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Net_Margin_Pct'].iloc[0]

print(f"\n{latest_year} Net Income Margin: {current_margin:.1f}%")

# Generate calculation display
income_fmt = format_number_compact(current_net_income)
revenue_fmt = format_number_compact(current_revenue)
calculation_display = f"Net income (${income_fmt}) ÷ Total revenue (${revenue_fmt})"

print(f"\nCalculation Display:")
print(f"  {calculation_display}")

# Verify calculation
verified_margin = (current_net_income / current_revenue) * 100
print(f"\nVerification:")
print(f"  Stored Margin: {current_margin:.6f}%")
print(f"  Calculated Margin: {verified_margin:.6f}%")
print(f"  Match: {abs(current_margin - verified_margin) < 0.0001}")

CARD #1: NET INCOME MARGIN

Latest Year: 2023

Net Income Margin - 5 Year History:
   Fiscal_Year  Net_Income  Total_Revenue  Net_Margin_Pct
0         2023         NaN            NaN             NaN
1         2021         NaN            NaN             NaN

2023 Net Income Margin: nan%

Calculation Display:
  Net income ($0) ÷ Total revenue ($0)

Verification:
  Stored Margin: nan%
  Calculated Margin: nan%
  Match: False


## 5. Card #2: Days in AR

In [84]:
print("=" * 100)
print("CARD #2: DAYS IN AR")
print("=" * 100)

# Extract relevant columns
ar_data = kpi_data[['Fiscal_Year', 'Accounts_Receivable', 'Total_Revenue', 'AR_Days']].copy()
ar_data = ar_data.sort_values('Fiscal_Year', ascending=False)

print("\nDays in AR - 5 Year History:")
print(ar_data)

# Get current year values
current_ar = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Accounts_Receivable'].iloc[0]
current_ar_days = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['AR_Days'].iloc[0]

print(f"\n{latest_year} Days in AR: {current_ar_days:.1f} days")

# Verify calculation
verified_ar_days = current_ar / (current_revenue / 365)
print(f"\nVerification:")
print(f"  Stored AR Days: {current_ar_days:.6f}")
print(f"  Calculated AR Days: {verified_ar_days:.6f}")
print(f"  Match: {abs(current_ar_days - verified_ar_days) < 0.0001}")

CARD #2: DAYS IN AR

Days in AR - 5 Year History:
   Fiscal_Year  Accounts_Receivable  Total_Revenue  AR_Days
0         2023            5532143.0            NaN      NaN
1         2021            3789301.0            NaN      NaN

2023 Days in AR: nan days

Verification:
  Stored AR Days: nan
  Calculated AR Days: nan
  Match: False


## 6. Card #3: Operating Expense per Adjusted Discharge

In [85]:
print("=" * 100)
print("CARD #3: OPERATING EXPENSE PER ADJUSTED DISCHARGE")
print("=" * 100)

# Note: This KPI requires additional data from worksheets
# For now, we'll show what's available in the main KPI table

print("\nNote: This KPI requires Adjusted Discharge data from worksheet S-3")
print("The calculation is: Total Operating Expenses ÷ Adjusted Discharges")

# Show operating expenses
oe_data = kpi_data[['Fiscal_Year', 'Total_Operating_Expenses']].copy()
oe_data = oe_data.sort_values('Fiscal_Year', ascending=False)

print("\nTotal Operating Expenses - 5 Year History:")
print(oe_data)

CARD #3: OPERATING EXPENSE PER ADJUSTED DISCHARGE

Note: This KPI requires Adjusted Discharge data from worksheet S-3
The calculation is: Total Operating Expenses ÷ Adjusted Discharges

Total Operating Expenses - 5 Year History:
   Fiscal_Year  Total_Operating_Expenses
0         2023                       NaN
1         2021                       NaN


## 7. Card #4: Medicare CCR

In [86]:
print("=" * 100)
print("CARD #4: MEDICARE COST-TO-CHARGE RATIO")
print("=" * 100)

print("\nNote: Medicare CCR data comes from worksheet S-10")
print("This KPI is computed separately from the main KPI table")

CARD #4: MEDICARE COST-TO-CHARGE RATIO

Note: Medicare CCR data comes from worksheet S-10
This KPI is computed separately from the main KPI table


## 8. Card #5: Bad Debt + Charity %

In [87]:
print("=" * 100)
print("CARD #5: BAD DEBT + CHARITY %")
print("=" * 100)

print("\nNote: Bad Debt and Charity Care data comes from worksheet G-3")
print("This KPI is computed separately from the main KPI table")

CARD #5: BAD DEBT + CHARITY %

Note: Bad Debt and Charity Care data comes from worksheet G-3
This KPI is computed separately from the main KPI table


## 9. Card #6: Current Ratio

In [88]:
print("=" * 100)
print("CARD #6: CURRENT RATIO")
print("=" * 100)

# Extract relevant columns
cr_data = kpi_data[['Fiscal_Year', 'Current_Assets', 'Current_Liabilities', 'Current_Ratio']].copy()
cr_data = cr_data.sort_values('Fiscal_Year', ascending=False)

print("\nCurrent Ratio - 5 Year History:")
print(cr_data)

# Get current year values
current_assets = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Current_Assets'].iloc[0]
current_liabilities = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Current_Liabilities'].iloc[0]
current_ratio = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Current_Ratio'].iloc[0]

print(f"\n{latest_year} Current Ratio: {current_ratio:.2f}")

# Generate calculation display
assets_fmt = format_number_compact(current_assets)
liabilities_fmt = format_number_compact(current_liabilities)
calculation_display = f"Current assets (${assets_fmt}) ÷ Current liabilities (${liabilities_fmt})"

print(f"\nCalculation Display:")
print(f"  {calculation_display}")

# Verify calculation
verified_ratio = current_assets / current_liabilities
print(f"\nVerification:")
print(f"  Stored Ratio: {current_ratio:.6f}")
print(f"  Calculated Ratio: {verified_ratio:.6f}")
print(f"  Match: {abs(current_ratio - verified_ratio) < 0.0001}")

CARD #6: CURRENT RATIO

Current Ratio - 5 Year History:
   Fiscal_Year  Current_Assets  Current_Liabilities  Current_Ratio
0         2023      11146696.0           29438442.0       0.378644
1         2021       7711316.0           21855864.0       0.352826

2023 Current Ratio: 0.38

Calculation Display:
  Current assets ($11.1M) ÷ Current liabilities ($29.4M)

Verification:
  Stored Ratio: 0.378644
  Calculated Ratio: 0.378644
  Match: True


## 10. Benchmark Data Extraction

In [89]:
print("=" * 100)
print("BENCHMARK DATA")
print("=" * 100)

# Get all 4 benchmark levels
all_benchmarks = {
    'state_hospital_type': data_manager.get_benchmarks(test_ccn, latest_year, 'State_Hospital_Type'),
    'state': data_manager.get_benchmarks(test_ccn, latest_year, 'State'),
    'hospital_type': data_manager.get_benchmarks(test_ccn, latest_year, 'Hospital_Type'),
    'national': data_manager.get_benchmarks(test_ccn, latest_year, 'National')
}

print("\nBenchmark Levels Available:")
for level_name, bench_data in all_benchmarks.items():
    if bench_data:
        print(f"  {level_name}: {bench_data.get('provider_count', 0)} peers")
        print(f"    Group: {bench_data.get('group_name', 'N/A')}")

BENCHMARK DATA
  [DEBUG] State_Hospital_Type: Found 5 KPI benchmarks, Provider_Count=4
  [DEBUG] State: Found 5 KPI benchmarks, Provider_Count=4
  [DEBUG] Hospital_Type: Found 18 KPI benchmarks, Provider_Count=8
  [DEBUG] National: Found 20 KPI benchmarks, Provider_Count=112

Benchmark Levels Available:
  state_hospital_type: 4 peers
    Group: 01 - Psychiatric
  state: 4 peers
    Group: State 01
  hospital_type: 8 peers
    Group: Psychiatric
  national: 112 peers
    Group: National


## 11. Net Income Margin Benchmarks

In [90]:
print("\n" + "=" * 100)
print("NET INCOME MARGIN BENCHMARKS")
print("=" * 100)

# Look for Net_Margin_Pct in benchmarks (this is the database column name)
db_column = 'Net_Margin_Pct'

print(f"\nCurrent Hospital Value: {current_margin:.1f}%\n")

for level_name, bench_data in all_benchmarks.items():
    if bench_data:
        kpi_bench = bench_data.get('kpis', {}).get(db_column, {})
        if kpi_bench:
            p25 = kpi_bench.get('P25')
            median = kpi_bench.get('Median')
            p75 = kpi_bench.get('P75')
            
            print(f"{level_name.upper()}:")
            print(f"  Peers: {bench_data.get('provider_count', 0)}")
            print(f"  P25: {p25:.1f}%")
            print(f"  Median: {median:.1f}%")
            print(f"  P75: {p75:.1f}%")
            
            # Determine quartile
            if current_margin <= p25:
                quartile = "Bottom Quartile (Poor)"
            elif current_margin <= median:
                quartile = "Second Quartile (Fair)"
            elif current_margin <= p75:
                quartile = "Third Quartile (Good)"
            else:
                quartile = "Top Quartile (Excellent)"
            
            print(f"  Hospital Position: {quartile}")
            print()


NET INCOME MARGIN BENCHMARKS

Current Hospital Value: nan%

HOSPITAL_TYPE:
  Peers: 8
  P25: 63.1%
  Median: 68.5%
  P75: 70.2%
  Hospital Position: Top Quartile (Excellent)

NATIONAL:
  Peers: 112
  P25: 56.9%
  Median: 60.1%
  P75: 63.5%
  Hospital Position: Top Quartile (Excellent)



## 12. Current Ratio Benchmarks

In [91]:
print("\n" + "=" * 100)
print("CURRENT RATIO BENCHMARKS")
print("=" * 100)

db_column = 'Current_Ratio'

print(f"\nCurrent Hospital Value: {current_ratio:.2f}\n")

for level_name, bench_data in all_benchmarks.items():
    if bench_data:
        kpi_bench = bench_data.get('kpis', {}).get(db_column, {})
        if kpi_bench:
            p25 = kpi_bench.get('P25')
            median = kpi_bench.get('Median')
            p75 = kpi_bench.get('P75')
            
            print(f"{level_name.upper()}:")
            print(f"  Peers: {bench_data.get('provider_count', 0)}")
            print(f"  P25: {p25:.2f}")
            print(f"  Median: {median:.2f}")
            print(f"  P75: {p75:.2f}")
            
            # Determine quartile
            if current_ratio <= p25:
                quartile = "Bottom Quartile (Poor)"
            elif current_ratio <= median:
                quartile = "Second Quartile (Fair)"
            elif current_ratio <= p75:
                quartile = "Third Quartile (Good)"
            else:
                quartile = "Top Quartile (Excellent)"
            
            print(f"  Hospital Position: {quartile}")
            print()


CURRENT RATIO BENCHMARKS

Current Hospital Value: 0.38

STATE_HOSPITAL_TYPE:
  Peers: 4
  P25: 0.40
  Median: 1.30
  P75: 3.17
  Hospital Position: Bottom Quartile (Poor)

STATE:
  Peers: 4
  P25: 0.40
  Median: 1.30
  P75: 3.17
  Hospital Position: Bottom Quartile (Poor)

HOSPITAL_TYPE:
  Peers: 8
  P25: 1.32
  Median: 2.18
  P75: 3.94
  Hospital Position: Bottom Quartile (Poor)

NATIONAL:
  Peers: 112
  P25: 0.97
  Median: 1.66
  P75: 3.20
  Hospital Position: Bottom Quartile (Poor)



## 13. Trend Calculation

In [92]:
print("=" * 100)
print("TREND CALCULATIONS")
print("=" * 100)

# Net Income Margin Trend
nim_values = kpi_data['Net_Margin_Pct'].values
if len(nim_values) >= 2:
    current_value = nim_values[0]  # Most recent
    previous_value = nim_values[1]  # Previous year
    
    if not pd.isna(current_value) and not pd.isna(previous_value) and previous_value != 0:
        trend_pct = ((current_value - previous_value) / abs(previous_value)) * 100
        trend_direction = 'up' if trend_pct > 0 else ('down' if trend_pct < 0 else 'flat')
        
        print(f"\nNet Income Margin:")
        print(f"  Current Year: {current_value:.1f}%")
        print(f"  Previous Year: {previous_value:.1f}%")
        print(f"  Change: {trend_pct:+.1f}% ({trend_direction})")

# Current Ratio Trend
cr_values = kpi_data['Current_Ratio'].values
if len(cr_values) >= 2:
    current_value = cr_values[0]
    previous_value = cr_values[1]
    
    if not pd.isna(current_value) and not pd.isna(previous_value) and previous_value != 0:
        trend_pct = ((current_value - previous_value) / abs(previous_value)) * 100
        trend_direction = 'up' if trend_pct > 0 else ('down' if trend_pct < 0 else 'flat')
        
        print(f"\nCurrent Ratio:")
        print(f"  Current Year: {current_value:.2f}")
        print(f"  Previous Year: {previous_value:.2f}")
        print(f"  Change: {trend_pct:+.1f}% ({trend_direction})")

TREND CALCULATIONS

Current Ratio:
  Current Year: 0.38
  Previous Year: 0.35
  Change: +7.3% (up)


## 14. Summary: Complete Card Data for Net Income Margin

In [93]:
print("=" * 100)
print("COMPLETE CARD DATA: NET INCOME MARGIN")
print("=" * 100)

print(f"\nHospital: CCN {test_ccn}")
print(f"Hospital Type: {hospital_type}")
print(f"Latest Year: {latest_year}")

print(f"\n📊 KPI VALUE:")
print(f"  Net Income Margin: {current_margin:.1f}%")

print(f"\n📈 TREND:")
if len(nim_values) >= 2 and not pd.isna(nim_values[0]) and not pd.isna(nim_values[1]):
    trend_pct = ((nim_values[0] - nim_values[1]) / abs(nim_values[1])) * 100
    arrow = "↑" if trend_pct > 0 else "↓"
    print(f"  {arrow} {abs(trend_pct):.1f}% vs Last Year")

print(f"\n🧮 CALCULATION:")
print(f"  {calculation_display}")

print(f"\n📅 5-YEAR HISTORY:")
for idx, row in nim_data.iterrows():
    year = int(row['Fiscal_Year'])
    value = row['Net_Margin_Pct']
    print(f"  {year}: {value:.1f}%")

print(f"\n📊 BENCHMARKS:")
for level_name, bench_data in all_benchmarks.items():
    if bench_data:
        kpi_bench = bench_data.get('kpis', {}).get('Net_Margin_Pct', {})
        if kpi_bench:
            p25 = kpi_bench.get('P25')
            median = kpi_bench.get('Median')
            p75 = kpi_bench.get('P75')
            
            # Determine quartile
            if current_margin <= p25:
                position = "Poor ⚠️"
            elif current_margin <= median:
                position = "Fair ⚡"
            elif current_margin <= p75:
                position = "Good ⭐"
            else:
                position = "Excellent ✅"
            
            print(f"  {level_name}: P25={p25:.1f}%, Median={median:.1f}%, P75={p75:.1f}% → {position}")

print("\n" + "=" * 100)

COMPLETE CARD DATA: NET INCOME MARGIN

Hospital: CCN 014000
Hospital Type: Psychiatric
Latest Year: 2023

📊 KPI VALUE:
  Net Income Margin: nan%

📈 TREND:

🧮 CALCULATION:
  Current assets ($11.1M) ÷ Current liabilities ($29.4M)

📅 5-YEAR HISTORY:
  2023: nan%
  2021: nan%

📊 BENCHMARKS:
  hospital_type: P25=63.1%, Median=68.5%, P75=70.2% → Excellent ✅
  national: P25=56.9%, Median=60.1%, P75=63.5% → Excellent ✅



## 15. Test Different Hospitals

In [ ]:
# You can change this to test other hospitals
test_hospitals = ['310001', '310005', '310010']

print("Comparison across multiple hospitals:")
print("=" * 100)

comparison_data = []
for ccn in test_hospitals:
    try:
        kpi_data_temp = data_manager.calculate_kpis(ccn)
        if not kpi_data_temp.empty:
            latest = kpi_data_temp['Fiscal_Year'].max()
            latest_data = kpi_data_temp[kpi_data_temp['Fiscal_Year'] == latest].iloc[0]
            
            comparison_data.append({
                'CCN': ccn,
                'Year': int(latest),
                'Net_Income_Margin': f"{latest_data['Net_Margin_Pct']:.1f}%",
                'Current_Ratio': f"{latest_data['Current_Ratio']:.2f}",
                'AR_Days': f"{latest_data['AR_Days']:.1f}"
            })
    except Exception as e:
        print(f"Error loading CCN {ccn}: {e}")

if comparison_data:
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))

Comparison across multiple hospitals:


   CCN  Year Net_Income_Margin Current_Ratio AR_Days
310001  2024             57.0%          5.71     1.2
310005  2024             54.9%          1.55     1.0
310010  2024             55.4%          1.28     0.8


: 